In [ ]:
import mysql.connector.pooling
import pandas as pd

In [ ]:
# Create a connection pool to the MySQL database
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "db_agency",
}

connection_pool = mysql.connector.pooling.MySQLConnectionPool(pool_name="my_pool", pool_size=5, **db_config)


In [ ]:
connection = connection_pool.get_connection()
print(connection.is_connected())

In [ ]:
movies_query =  "SELECT * FROM movies;"
movies_df = pd.read_sql_query(movies_query, connection)
print(movies_df.keys())
print(movies_df.head())

In [ ]:
genres_query =  "SELECT mg.movie_id, g.genre FROM movie_genre mg JOIN genre g ON mg.genre_id = g.id;"
genre_df = pd.read_sql_query(genres_query, connection)
print(genre_df.keys())
print(genre_df.head())

In [ ]:
# Group by movie_id and aggregate genres into a list
grouped_genre_df = genre_df.groupby('movie_id')['genre'].agg(list).reset_index()
print(grouped_genre_df.head())

In [ ]:
grouped_genre_df['movie_id'] = grouped_genre_df['movie_id'].astype('int64')
merged_movies = movies_df.merge(grouped_genre_df, left_on='id', right_on='movie_id', how='left')
print(merged_movies.head())


In [ ]:
merged_movies.columns

In [ ]:
merged_movies = merged_movies[['id', 'title', 'genre', 'summary', 'release_date', 'duration', 'parental_guide', 'actors']]

In [ ]:
# Replace null values in 'actors' column with empty list
# merged_movies['actors'].fillna([], inplace=True)
# merged_movies.head()
merged_movies.fillna('', inplace=True)

In [ ]:
merged_movies['summary'] = merged_movies['summary'].apply(lambda x:x.split())
merged_movies['summary'] = merged_movies['summary'].apply(lambda x:[i.replace(" ", "") for i in x])
merged_movies['summary'].head()

In [ ]:
merged_movies['actors'] = merged_movies['actors'].apply(lambda x:x.split())
merged_movies['actors'] = merged_movies['actors'].apply(lambda x:[i.replace(" ", "") for i in x])
merged_movies['actors'].head()

In [ ]:
# Custom function to concatenate lists
def concatenate_lists(row, columns=[]):
    concatenated = []
    for column in columns:
        concatenated.extend(row[column])
    return concatenated

# Define the columns you want to concatenate
columns_to_concat = ['genre', 'summary', 'actors']
merged_movies['tags'] = merged_movies.apply(lambda x: concatenate_lists(x, columns_to_concat), axis=1)

merged_movies['tags'].head()


In [ ]:
import ast

def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

print(ast.literal_eval('[]'))

In [ ]:
new_df = merged_movies[['id', 'title', 'tags', 'release_date', 'duration', 'parental_guide']]
new_df['tags'] = new_df['tags'].apply(lambda x:' '.join(x))
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Matrix based on count vectorizer and tfidf vectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
# vectors = cv.fit_transform(new_df['tags']).toarray()
vectors = tfidf.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.fit_transform(new_df['tags']).toarray().shape
len(cv.get_feature_names_out())

In [ ]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)


In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(vectors).shape

In [ ]:
similarity = cosine_similarity(vectors)
similarity[0].shape

In [ ]:
def recommend(movie):
    # movie_index = new_df[new_df['title']==movie].index[0]
    # modification to make the search case-insesitive
    movie_index = new_df[new_df['title'].apply(lambda x:x.lower())==movie.lower()].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend(new_df.iloc[5].title)